ELT

Extract y Load

In [23]:
# Abres un cliente de S3
import awswrangler as wr
import boto3
session = boto3.Session(profile_name='arquitectura')
s3 = session.client('s3')

In [24]:
#Llamar cliente de glue
from botocore.config import Config
my_config = Config(
    region_name = 'us-east-1'
)

glue = boto3.client('glue', config=my_config)

In [4]:
response = glue.create_database(
    DatabaseInput={
        'Name': 'econ'
    },
)
response
#No salio por las credenciales pero creamos la bd directo en Glue

NoCredentialsError: Unable to locate credentials

In [6]:
#Crear tabla inflacion
query = '''
    CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`inflacion` (
    `fecha` date,
    `inflacion` float
    ) COMMENT "Inflacion indicador 628194 de INEGI."
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
    WITH SERDEPROPERTIES ('field.delim' = ',')
    STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 's3://itam-analytics-enriquenava/tarea6/raw/inflacion/'
    TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

In [7]:
wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False, 
    boto3_session=session
)

""


In [25]:
#Crear tabla tipo de cambio
query = '''
    CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`tipo_de_cambio` (
    `fecha` date,
    `tipo_de_cambio` float
    ) COMMENT "Tipo de cambio indicador SF17906 de BANXICO."
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
    WITH SERDEPROPERTIES ('field.delim' = ',')
    STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 's3://itam-analytics-enriquenava/tarea6/raw/tipocambio/'
    TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

In [26]:
wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False, 
    boto3_session=session
)

""


In [27]:
#Crear tabla tasa de interes
query = '''
    CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`tasa_de_interes` (
    `fecha` date,
    `tasa_de_interes` float
    ) COMMENT "Tasa de Interes indicador SF282 de BANXICO."
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
    WITH SERDEPROPERTIES ('field.delim' = ',')
    STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 's3://itam-analytics-enriquenava/tarea6/raw/tasainteres/'
    TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

In [28]:
wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False, 
    boto3_session=session
)

""


Transform

In [42]:
query = '''
CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`unificadas` (
    `fecha` DATE,
    `inflacion` DECIMAL(10,2),
    `tasa_de_interes` DECIMAL(10,2),
    `tipo_de_cambio` DECIMAL(10,2)
)
LOCATION 's3://itam-analytics-enriquenava/tarea6/unificadas/'
;
'''

In [43]:
wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False,
    boto3_session=session
)

""


In [46]:
query = '''
INSERT INTO econ.unificadas (fecha, inflacion, tasa_de_interes, tipo_de_cambio)
SELECT 
    i.fecha, 
    CAST(i.inflacion AS DECIMAL(10, 2)),
    CAST(ti.tasa_de_interes AS DECIMAL(10, 2)),
    CAST(tc.tipo_de_cambio AS DECIMAL(10, 2))
FROM econ.unificadas as i
INNER JOIN econ.tasa_de_interes AS ti ON i.fecha = ti.fecha
INNER JOIN econ.tipo_de_cambio AS tc ON i.fecha = tc.fecha;
'''

In [47]:
wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False,
    boto3_session=session
)

""
